In [6]:
%pip install torch transformers peft accelerate datasets tqdm scikit-learn


  Using cached peft-0.17.1-py3-none-any.whl.metadata (14 kB)
  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 7.4 MB/s  0:01:41 eta 0:00:010:00:03m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 8.0 MB/s  0:01:07 eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 12.4 MB/s  0:00:00 eta 0:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.0 MB/s  0:00:09 eta 0:00:010:01:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 6.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 8.1 MB/s  0:01:19 eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 9.0 MB/s  0:00:21 eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 8.9 MB/s  0:00:07 eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5

In [ ]:
#%pip install peft --upgrade
import peft
#print("PEFT version:", peft.__version__)


In [17]:
import os
import math
import random
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import GroupKFold
from transformers import AutoTokenizer, AutoModel
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

notebook_dir = os.getcwd()
#print(f"Current working directory: {notebook_dir}")
DATA_DIR = os.path.join(notebook_dir, "data")

train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
sample = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
TEST_CSV  = os.path.join(DATA_DIR, "test.csv")

BASE_MODEL_NAME = "roberta-base"   # can downgrade to "roberta-base" if VRAM explodes
MAX_LEN = 512                       # keep this reasonable for your GPU
BATCH_SIZE = 2                      # we use grad accumulation instead of big batch
GRAD_ACCUM_STEPS = 8                # effective batch_size ~= 16
LR = 2e-5
EPOCHS = 2
SEED = 42
N_FOLDS = 5
OUT_DIR = "reward_teacher_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


Using device: cuda


In [18]:
df = pd.read_csv(TRAIN_CSV)

# sanity: make one label column 0/1/2
def row_label(r):
    if r["winner_model_a"] == 1:
        return 0  # A preferred
    elif r["winner_model_b"] == 1:
        return 1  # B preferred
    else:
        return 2  # tie

df["label"] = df.apply(row_label, axis=1)

# We also keep lengths if you want analysis later
df["len_a"] = df["response_a"].astype(str).str.len()
df["len_b"] = df["response_b"].astype(str).str.len()

# groups for CV: group by prompt (hash to avoid giant strings)
df["prompt_group"] = df["prompt"].astype(str).apply(lambda x: hash(x) % (10**9))
print(df[["label","len_a","len_b"]].head())
print("class counts:\n", df["label"].value_counts())


   label  len_a  len_b
0      0   4538   1206
1      1   3114   3649
2      2    921   1835
3      0   3182   1562
4      1   1300    772
class counts:
 label
0    20064
1    19652
2    17761
Name: count, dtype: int64


In [19]:
class PairwisePrefDataset(Dataset):
    def __init__(self, df_fold, tokenizer, max_len=512):
        self.df = df_fold.reset_index(drop=True)
        self.tok = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def encode_pair(self, prompt, resp):
        # We'll just concat prompt + </s> + resp style for RoBERTa:
        text = f"Prompt: {prompt}\nAnswer:\n{resp}"
        return self.tok(
            text,
            truncation=True,
            max_length=self.max_len,
            padding="max_length",
            return_tensors="pt"
        )

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc_a = self.encode_pair(row["prompt"], row["response_a"])
        enc_b = self.encode_pair(row["prompt"], row["response_b"])
        label = row["label"]
        return {
            "input_ids_a": enc_a["input_ids"].squeeze(0),
            "attention_mask_a": enc_a["attention_mask"].squeeze(0),
            "input_ids_b": enc_b["input_ids"].squeeze(0),
            "attention_mask_b": enc_b["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
            "row_id": torch.tensor(row["id"], dtype=torch.long),
        }


In [ ]:
#draft one dont need to run now
'''from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
from peft import LoraConfig, get_peft_model

class RewardScorer(nn.Module):
    def __init__(self, base_model_name):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(base_model_name)
        self.config = self.backbone.config   # <--- this fixes the AttributeError
        hidden_size = self.config.hidden_size
        self.score_head = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        cls_state = out.last_hidden_state[:, 0, :]   # CLS embedding
        score = self.score_head(cls_state)           # [batch, 1]
        return score.squeeze(-1)                     # [batch]

def build_lora_reward_model(base_model_name):
    base = RewardScorer(base_model_name)

    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=[
            "query", "key", "value", "dense", "out_proj"
        ],  # common layers in RoBERTa’s self-attn + FFN
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_CLS"
    )

    model = get_peft_model(base, peft_config)
    return model

# Example usage
BASE_MODEL_NAME = "roberta-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
model = build_lora_reward_model(BASE_MODEL_NAME).to(DEVICE)

print(" Model ready on", DEVICE)'''


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 Model ready on cpu


In [20]:
# ===============================================================
# ✅ FINAL REWARD SCORER + LoRA BUILDER (100% stable)
# ===============================================================

from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn
from peft import LoraConfig, get_peft_model


# ---------------------------------------------------------------
# RewardScorer — scalar scoring head for reward modeling
# ---------------------------------------------------------------
class RewardScorer(nn.Module):
    def __init__(self, base_model_name):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(base_model_name)
        self.config = self.backbone.config
        hidden_size = self.config.hidden_size
        self.score_head = nn.Linear(hidden_size, 1)

        # ✅ Fix: remove dummy token_type_ids buffer from RoBERTa
        if hasattr(self.backbone.embeddings, "token_type_ids"):
            del self.backbone.embeddings.token_type_ids

    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        # ✅ Clean up unexpected kwargs from PEFT
        kwargs.pop("labels", None)
        kwargs.pop("output_attentions", None)
        kwargs.pop("output_hidden_states", None)
        kwargs.pop("return_dict", None)

        # ✅ token_type_ids=None ensures no buffer expansion bug
        out = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None,
            **kwargs
        )

        # CLS token embedding (RoBERTa uses <s> as CLS)
        cls_state = out.last_hidden_state[:, 0, :]
        score = self.score_head(cls_state)
        return score.squeeze(-1)  # shape: [batch]


# ---------------------------------------------------------------
# Build model + apply LoRA adapters
# ---------------------------------------------------------------
def build_lora_reward_model(base_model_name):
    base = RewardScorer(base_model_name)

    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["query", "key", "value", "dense", "out_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="FEATURE_EXTRACTION",  # prevents PEFT from adding 'labels'
    )

    model = get_peft_model(base, peft_config)
    return model


# ---------------------------------------------------------------
# Example usage / sanity check
# ---------------------------------------------------------------
BASE_MODEL_NAME = "roberta-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
model = build_lora_reward_model(BASE_MODEL_NAME).to(DEVICE)

if hasattr(model, "print_trainable_parameters"):
    model.print_trainable_parameters()

print(f"✅ Model ready on {DEVICE}")

# Quick smoke test
inputs = tokenizer(
    ["hello world", "great job!"],
    padding=True,
    truncation=True,
    return_tensors="pt"
).to(DEVICE)

with torch.no_grad():
    scores = model(**inputs)
    print("Scores:", scores)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 2,678,784 || all params: 127,325,185 || trainable%: 2.1039
✅ Model ready on cuda
Scores: tensor([0.1878, 0.1746], device='cuda:0')


In [21]:
def pref_loss(scores_a, scores_b, labels, margin=0.5, tie_band=0.2):
    # scores_a, scores_b: [batch]
    # labels: [batch] in {0,1,2}
    loss_list = []

    # A wins -> sA >= sB + margin
    mask_a = (labels == 0)
    if mask_a.any():
        diff = scores_b[mask_a] + margin - scores_a[mask_a]
        # want diff <= 0
        loss_a = torch.clamp(diff, min=0).mean()
        loss_list.append(loss_a)

    # B wins -> sB >= sA + margin
    mask_b = (labels == 1)
    if mask_b.any():
        diff = scores_a[mask_b] + margin - scores_b[mask_b]
        loss_b = torch.clamp(diff, min=0).mean()
        loss_list.append(loss_b)

    # Tie -> |sA - sB| <= tie_band
    mask_t = (labels == 2)
    if mask_t.any():
        diff = torch.abs(scores_a[mask_t] - scores_b[mask_t]) - tie_band
        loss_t = torch.clamp(diff, min=0).mean()
        loss_list.append(loss_t)

    if len(loss_list) == 0:
        return torch.tensor(0.0, device=scores_a.device, requires_grad=True)
    return torch.stack(loss_list).mean()


In [ ]:
gkf = GroupKFold(n_splits=N_FOLDS)
folds = []
for fold_id, (tr_idx, va_idx) in enumerate(gkf.split(df, groups=df["prompt_group"])):
    df.loc[va_idx, "fold"] = fold_id
df["fold"] = df["fold"].astype(int)

def train_one_fold(fold_id):
    print(f"\n===== FOLD {fold_id} =====")
    df_tr = df[df["fold"] != fold_id].reset_index(drop=True)
    df_va = df[df["fold"] == fold_id].reset_index(drop=True)

    train_ds = PairwisePrefDataset(df_tr, tokenizer, max_len=MAX_LEN)
    va_ds    = PairwisePrefDataset(df_va, tokenizer, max_len=MAX_LEN)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    va_loader    = DataLoader(va_ds, batch_size=BATCH_SIZE, shuffle=False)

    model = build_lora_reward_model(BASE_MODEL_NAME).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

    # simple linear warmup + cosine cooldown
    total_steps = math.ceil(len(train_loader) / GRAD_ACCUM_STEPS) * EPOCHS
    warmup_steps = min(100, total_steps // 10)

    def lr_lambda(step):
        if step < warmup_steps:
            return step / max(1, warmup_steps)
        # cosine decay after warmup
        progress = (step - warmup_steps) / max(1, (total_steps - warmup_steps))
        return 0.5 * (1 + math.cos(math.pi * progress))

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    global_step = 0
    model.train()
    for epoch in range(EPOCHS):
        optimizer.zero_grad()
        running_loss = 0.0
        for step, batch in enumerate(tqdm(train_loader, desc=f"Fold {fold_id} Epoch {epoch}")):
            input_ids_a = batch["input_ids_a"].to(DEVICE)
            attn_a      = batch["attention_mask_a"].to(DEVICE)
            input_ids_b = batch["input_ids_b"].to(DEVICE)
            attn_b      = batch["attention_mask_b"].to(DEVICE)
            labels      = batch["label"].to(DEVICE)

            scores_a = model(input_ids_a, attn_a)
            scores_b = model(input_ids_b, attn_b)

            loss = pref_loss(scores_a, scores_b, labels)
            loss = loss / GRAD_ACCUM_STEPS
            loss.backward()
            running_loss += loss.item()

            if (step + 1) % GRAD_ACCUM_STEPS == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                global_step += 1

        print(f"Epoch {epoch} avg loss (divided): {running_loss/len(train_loader):.4f}")

    # after training, eval on val fold to get a feel
    model.eval()
    all_scores = []
    all_labels = []
    all_ids    = []
    with torch.no_grad():
        for batch in tqdm(va_loader, desc=f"Eval fold {fold_id}"):
            input_ids_a = batch["input_ids_a"].to(DEVICE)
            attn_a      = batch["attention_mask_a"].to(DEVICE)
            input_ids_b = batch["input_ids_b"].to(DEVICE)
            attn_b      = batch["attention_mask_b"].to(DEVICE)
            labels      = batch["label"].to(DEVICE)
            row_ids     = batch["row_id"].cpu().numpy()

            scores_a = model(input_ids_a, attn_a).cpu().numpy()
            scores_b = model(input_ids_b, attn_b).cpu().numpy()

            all_scores.append(np.stack([scores_a, scores_b], axis=1))  # shape [batch,2]
            all_labels.append(labels.cpu().numpy())
            all_ids.append(row_ids)

    all_scores = np.concatenate(all_scores, axis=0)   # [N_val,2]
    all_labels = np.concatenate(all_labels, axis=0)   # [N_val]
    all_ids    = np.concatenate(all_ids, axis=0)      # [N_val]

    # save checkpoint
    fold_dir = os.path.join(OUT_DIR, f"fold_{fold_id}")
    os.makedirs(fold_dir, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(fold_dir, "reward_teacher_roberta.pt"))

    # we also save the val fold scores for later calibration/logit building
    oof_df = pd.DataFrame({
        "id": all_ids,
        "score_a": all_scores[:,0],
        "score_b": all_scores[:,1],
        "label": all_labels,
        "fold": fold_id
    })
    oof_df.to_csv(os.path.join(fold_dir, "oof_scores.csv"), index=False)

    return oof_df

all_oof = []
for f in range(N_FOLDS):
    oof_df = train_one_fold(f)
    all_oof.append(oof_df)

all_oof = pd.concat(all_oof, ignore_index=True)
all_oof.to_csv(os.path.join(OUT_DIR, "all_oof_scores.csv"), index=False)
print("Saved all OOF scores to", os.path.join(OUT_DIR, "all_oof_scores.csv"))


Fold 1 Epoch 1: 100%|██████████| 22991/22991 [53:41<00:00,  7.14it/s]


Epoch 1 avg loss (divided): 0.0377


Eval fold 1: 100%|██████████| 5748/5748 [05:17<00:00, 18.08it/s]



===== FOLD 2 =====


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Fold 2 Epoch 0: 100%|██████████| 22991/22991 [53:35<00:00,  7.15it/s] 


Epoch 0 avg loss (divided): 0.0395


Fold 2 Epoch 1:  95%|█████████▍| 21820/22991 [50:51<02:42,  7.22it/s] 

In [ ]:
# just repeated section of the code to continue with training 
# this has checkpoints logged so keep this code 
gkf = GroupKFold(n_splits=N_FOLDS)
folds = []
for fold_id, (tr_idx, va_idx) in enumerate(gkf.split(df, groups=df["prompt_group"])):
    df.loc[va_idx, "fold"] = fold_id
df["fold"] = df["fold"].astype(int)
# ==========================================================
# TRAINING LOOP — now resume-safe (per-epoch save + skip done folds)
# ==========================================================

def train_one_fold(fold_id):
    print(f"\n===== FOLD {fold_id} =====")
    df_tr = df[df["fold"] != fold_id].reset_index(drop=True)
    df_va = df[df["fold"] == fold_id].reset_index(drop=True)

    train_ds = PairwisePrefDataset(df_tr, tokenizer, max_len=MAX_LEN)
    va_ds    = PairwisePrefDataset(df_va, tokenizer, max_len=MAX_LEN)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    va_loader    = DataLoader(va_ds, batch_size=BATCH_SIZE, shuffle=False)

    model = build_lora_reward_model(BASE_MODEL_NAME).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

    # scheduler: linear warmup + cosine decay
    total_steps = math.ceil(len(train_loader) / GRAD_ACCUM_STEPS) * EPOCHS
    warmup_steps = min(100, total_steps // 10)

    def lr_lambda(step):
        if step < warmup_steps:
            return step / max(1, warmup_steps)
        progress = (step - warmup_steps) / max(1, (total_steps - warmup_steps))
        return 0.5 * (1 + math.cos(math.pi * progress))

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    global_step = 0
    model.train()
    for epoch in range(EPOCHS):
        optimizer.zero_grad()
        running_loss = 0.0
        for step, batch in enumerate(tqdm(train_loader, desc=f"Fold {fold_id} Epoch {epoch}")):
            input_ids_a = batch["input_ids_a"].to(DEVICE)
            attn_a      = batch["attention_mask_a"].to(DEVICE)
            input_ids_b = batch["input_ids_b"].to(DEVICE)
            attn_b      = batch["attention_mask_b"].to(DEVICE)
            labels      = batch["label"].to(DEVICE)

            scores_a = model(input_ids_a, attn_a)
            scores_b = model(input_ids_b, attn_b)

            loss = pref_loss(scores_a, scores_b, labels)
            loss = loss / GRAD_ACCUM_STEPS
            loss.backward()
            running_loss += loss.item()

            if (step + 1) % GRAD_ACCUM_STEPS == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                global_step += 1

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch} avg loss (divided): {avg_loss:.4f}")

        #  Save checkpoint after each epoch
        fold_dir = os.path.join(OUT_DIR, f"fold_{fold_id}")
        os.makedirs(fold_dir, exist_ok=True)
        ckpt_path = os.path.join(fold_dir, f"epoch_{epoch}.pt")
        torch.save(model.state_dict(), ckpt_path)
        print(f" Saved checkpoint: {ckpt_path}")

    # ============================
    # Evaluation on validation set
    # ============================
    model.eval()
    all_scores, all_labels, all_ids = [], [], []
    with torch.no_grad():
        for batch in tqdm(va_loader, desc=f"Eval fold {fold_id}"):
            input_ids_a = batch["input_ids_a"].to(DEVICE)
            attn_a      = batch["attention_mask_a"].to(DEVICE)
            input_ids_b = batch["input_ids_b"].to(DEVICE)
            attn_b      = batch["attention_mask_b"].to(DEVICE)
            labels      = batch["label"].to(DEVICE)
            row_ids     = batch["row_id"].cpu().numpy()

            scores_a = model(input_ids_a, attn_a).cpu().numpy()
            scores_b = model(input_ids_b, attn_b).cpu().numpy()

            all_scores.append(np.stack([scores_a, scores_b], axis=1))
            all_labels.append(labels.cpu().numpy())
            all_ids.append(row_ids)

    all_scores = np.concatenate(all_scores, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    all_ids    = np.concatenate(all_ids, axis=0)

    # save final fold model + validation results
    torch.save(model.state_dict(), os.path.join(fold_dir, "reward_teacher_roberta.pt"))
    oof_df = pd.DataFrame({
        "id": all_ids,
        "score_a": all_scores[:, 0],
        "score_b": all_scores[:, 1],
        "label": all_labels,
        "fold": fold_id
    })
    oof_df.to_csv(os.path.join(fold_dir, "oof_scores.csv"), index=False)
    return oof_df


# ==========================================================
# MAIN LOOP — skip completed folds automatically
# ==========================================================
all_oof = []
for f in range(N_FOLDS):
    fold_dir = os.path.join(OUT_DIR, f"fold_{f}")
    final_ckpt = os.path.join(fold_dir, "reward_teacher_roberta.pt")
    if os.path.exists(final_ckpt):
        print(f" Fold {f} already completed — skipping.")
        continue

    oof_df = train_one_fold(f)
    all_oof.append(oof_df)

if all_oof:
    all_oof = pd.concat(all_oof, ignore_index=True)
    all_oof.to_csv(os.path.join(OUT_DIR, "all_oof_scores.csv"), index=False)
    print(" Saved all OOF scores to", os.path.join(OUT_DIR, "all_oof_scores.csv"))


 Fold 0 already completed — skipping.
 Fold 1 already completed — skipping.

===== FOLD 2 =====


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Fold 2 Epoch 0: 100%|██████████| 22991/22991 [1:10:47<00:00,  5.41it/s]


Epoch 0 avg loss (divided): 0.0397
 Saved checkpoint: reward_teacher_outputs/fold_2/epoch_0.pt


Fold 2 Epoch 1: 100%|██████████| 22991/22991 [1:10:54<00:00,  5.40it/s]


Epoch 1 avg loss (divided): 0.0381
 Saved checkpoint: reward_teacher_outputs/fold_2/epoch_1.pt


...

Fold 3 Epoch 1: 100%|██████████| 22991/22991 [1:03:44<00:00,  6.01it/s]


Epoch 1 avg loss (divided): 0.0375
 Saved checkpoint: reward_teacher_outputs/fold_3/epoch_1.pt


Eval fold 3: 100%|██████████| 5748/5748 [06:49<00:00, 14.05it/s]



===== FOLD 4 =====


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Fold 4 Epoch 0:  39%|███▊      | 8875/22991 [24:35<38:46,  6.07it/s]  

In [ ]:
import scipy.optimize as opt
import torch.nn.functional as F

# map label int -> onehot [A,B,Tie]
def label_to_onehot(y):
    out = np.zeros((len(y),3), dtype=np.float32)
    for i,lab in enumerate(y):
        out[i, lab] = 1.0
    return out

oof = pd.read_csv(os.path.join(OUT_DIR, "all_oof_scores.csv"))

y_true = label_to_onehot(oof["label"].values)
sa = oof["score_a"].values
sb = oof["score_b"].values

def probs_from_scores(sa, sb, temp, tie_alpha):
    # Bradley-Terry-ish:
    # pA_raw = exp(sa/temp)
    # pB_raw = exp(sb/temp)
    # tie_raw = exp(-abs(sa-sb)*tie_alpha)
    pA_raw = np.exp(sa / temp)
    pB_raw = np.exp(sb / temp)
    tie_raw = np.exp(-np.abs(sa - sb) * tie_alpha)

    denom = pA_raw + pB_raw + tie_raw + 1e-9
    pA = pA_raw / denom
    pB = pB_raw / denom
    pT = tie_raw / denom
    return np.stack([pA,pB,pT], axis=1)

def logloss(params):
    temp = np.exp(params[0])          # >0
    tie_alpha = np.exp(params[1])     # >0
    pred = probs_from_scores(sa, sb, temp, tie_alpha)
    # clip for stability
    pred = np.clip(pred, 1e-7, 1-1e-7)
    return -np.mean((y_true * np.log(pred)).sum(axis=1))

res = opt.minimize(logloss, x0=[0.0, 0.0], method="Nelder-Mead")
print("opt result:", res)

best_temp = math.exp(res.x[0])
best_tie_alpha = math.exp(res.x[1])
print("best_temp =", best_temp, " best_tie_alpha =", best_tie_alpha)

# save calibration params
calib_path = os.path.join(OUT_DIR, "calibration_params.json")
import json
with open(calib_path, "w") as f:
    json.dump({"temp": best_temp, "tie_alpha": best_tie_alpha}, f)
print("Saved", calib_path)


In [ ]:
import json
from glob import glob

with open(os.path.join(OUT_DIR, "calibration_params.json")) as f:
    calib = json.load(f)
TEMP = calib["temp"]
TIE_ALPHA = calib["tie_alpha"]

def get_model_for_fold(fold_id):
    m = build_lora_reward_model(BASE_MODEL_NAME).to(DEVICE)
    state_path = os.path.join(OUT_DIR, f"fold_{fold_id}", "reward_teacher_roberta.pt")
    m.load_state_dict(torch.load(state_path, map_location=DEVICE))
    m.eval()
    return m

def score_pairs_df(df_any, max_len=MAX_LEN):
    ds = PairwisePrefDataset(df_any, tokenizer, max_len=max_len)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False)
    fold_scores_a = []
    fold_scores_b = []
    with torch.no_grad():
        for fold_id in range(N_FOLDS):
            model_f = get_model_for_fold(fold_id)
            sa_list, sb_list = [], []
            for batch in tqdm(loader, desc=f"Infer fold {fold_id}"):
                input_ids_a = batch["input_ids_a"].to(DEVICE)
                attn_a      = batch["attention_mask_a"].to(DEVICE)
                input_ids_b = batch["input_ids_b"].to(DEVICE)
                attn_b      = batch["attention_mask_b"].to(DEVICE)

                scores_a = model_f(input_ids_a, attn_a).cpu().numpy()
                scores_b = model_f(input_ids_b, attn_b).cpu().numpy()
                sa_list.append(scores_a)
                sb_list.append(scores_b)

            sa_all = np.concatenate(sa_list)
            sb_all = np.concatenate(sb_list)
            fold_scores_a.append(sa_all)
            fold_scores_b.append(sb_all)

    # average across folds
    sa_mean = np.mean(np.stack(fold_scores_a, axis=0), axis=0)
    sb_mean = np.mean(np.stack(fold_scores_b, axis=0), axis=0)
    return sa_mean, sb_mean

# score train
sa_train, sb_train = score_pairs_df(df)
# score test
df_test = pd.read_csv(TEST_CSV)
df_test["label"] = -1  # dummy
df_test["prompt_group"] = df_test["prompt"].astype(str).apply(lambda x: hash(x) % (10**9))
sa_test, sb_test = score_pairs_df(df_test)

def probs_from_scores_numpy(sa, sb, temp, tie_alpha):
    pA_raw = np.exp(sa / temp)
    pB_raw = np.exp(sb / temp)
    tie_raw = np.exp(-np.abs(sa - sb) * tie_alpha)
    denom = pA_raw + pB_raw + tie_raw + 1e-9
    pA = pA_raw / denom
    pB = pB_raw / denom
    pT = tie_raw / denom
    return np.stack([pA,pB,pT], axis=1)

probs_train = probs_from_scores_numpy(sa_train, sb_train, TEMP, TIE_ALPHA)
probs_test  = probs_from_scores_numpy(sa_test,  sb_test,  TEMP, TIE_ALPHA)

teacher_train_out = pd.DataFrame({
    "id": df["id"].values,
    "pA": probs_train[:,0],
    "pB": probs_train[:,1],
    "pTie": probs_train[:,2],
    "label": df["label"].values
})
teacher_test_out = pd.DataFrame({
    "id": df_test["id"].values,
    "pA": probs_test[:,0],
    "pB": probs_test[:,1],
    "pTie": probs_test[:,2],
})

teacher_train_out.to_csv(os.path.join(OUT_DIR, "teacher_logits_train.csv"), index=False)
teacher_test_out.to_csv(os.path.join(OUT_DIR, "teacher_logits_test.csv"), index=False)

print("Saved distillation targets:")
print(" -", os.path.join(OUT_DIR, "teacher_logits_train.csv"))
print(" -", os.path.join(OUT_DIR, "teacher_logits_test.csv"))
